In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE128387"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE128387"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE128387.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE128387.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE128387.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from children and adolescents treated with Fluoxetine"
!Series_summary	"It is well-known that between  40 and 50% of patients taking antidepressants do not respond to treatment or relapse. Genome wide gene expression studies can help us to understand better the response to antidepressants, revealing the effects of both genetic background and environmental/epigenetic factors."
!Series_summary	"We used microarrays to detail the response to Fluoxetine in children and adolescents, analysing the expression just before intake of drug and 8 weeks after starting the treatment."
!Series_overall_design	"RNA extraction was done from blood of patients. Two samples of each patient were obtained, one previous to treatment and another 8 weeks later.  The samples with better quality were selected for hybridization and Affymetrix microarrays."
Sample Characteristics Dictionary:
{0: ['tissue: Blood'], 1: ['illness: Major Depressive Disorder'], 2: ['

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this study used Affymetrix microarrays
# to measure gene expression in blood samples, so gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary:
# Key 1 contains illness data (Major Depressive Disorder) - this is our trait
# Key 2 contains age data (ages 12-17)
# Key 3 contains gender data (female/male)
trait_row = 1
age_row = 2
gender_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert depression trait data to binary format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "major depressive disorder" in value:
        return 1  # Has depression
    return None  # Unknown or other condition

def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save Metadata - Initial Filtering
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Check if clinical data file exists before attempting to read it
clinical_data_path = f"{in_cohort_dir}/clinical_data.csv"
if os.path.exists(clinical_data_path):
    # Since trait_row is not None, we proceed with clinical feature extraction
    clinical_data = pd.read_csv(clinical_data_path)

    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)

    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
else:
    print(f"Clinical data file not found at: {clinical_data_path}")
    print("Skipping clinical feature extraction step.")


Clinical data file not found at: ../../input/GEO/Depression/GSE128387/clinical_data.csv
Skipping clinical feature extraction step.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE128387/GSE128387_series_matrix.txt.gz
Gene data shape: (48144, 32)
First 20 gene/probe identifiers:
Index(['16657436', '16657440', '16657445', '16657447', '16657450', '16657469',
       '16657473', '16657476', '16657480', '16657485', '16657489', '16657492',
       '16657502', '16657506', '16657509', '16657514', '16657527', '16657529',
       '16657534', '16657554'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (16657436, 16657440, etc.) appear to be numeric IDs, likely probe IDs
# from a microarray platform rather than standard human gene symbols.
# Human gene symbols would typically be alphanumeric like BRCA1, TP53, etc.
# Therefore, mapping to gene symbols will be required for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Examine a sample of the gene_assignment column to verify it contains gene symbols
print("\nSample from gene_assignment column (first entry):")
if 'gene_assignment' in gene_annotation.columns:
    sample_gene_assignment = gene_annotation['gene_assignment'].iloc[0]
    print(sample_gene_assignment)
    
    # Extract gene symbols from the sample to verify
    sample_symbols = extract_human_gene_symbols(sample_gene_assignment)
    print("\nExtracted gene symbols from sample:")
    print(sample_symbols)

# Define columns for gene mapping
prob_col = 'ID'
gene_col = 'gene_assignment'
print(f"\nWill use '{prob_col}' for probe IDs and '{gene_col}' for gene symbols in mapping step")

# Test extracting mapping data
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"\nMapping data shape: {mapping_data.shape}")
print("First few rows of mapping data:")
print(preview_df(mapping_data, n=3))



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'crosshyb_type', 'category', 'GB_ACC', 'SPOT_ID']
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DE

### Step 6: Gene Identifier Mapping

In [7]:
# Convert probe-level measurements to gene-level measurements by applying the gene mapping
# Apply the gene mapping to get a dataframe with genes as rows and samples as columns
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Ensure no empty symbols
gene_data = gene_data[gene_data.index.notnull() & (gene_data.index != '')]

# Normalize gene symbols to standardize and combine rows with the same gene
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print information about the transformed data
print(f"Gene expression data converted from probes to genes.")
print(f"Shape of gene expression data: {gene_data.shape}")
print(f"First 5 gene symbols:")
print(gene_data.index[:5])
print(f"First 5 columns (samples):")
print(gene_data.columns[:5])


Gene expression data converted from probes to genes.
Shape of gene expression data: (23274, 32)
First 5 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')
First 5 columns (samples):
Index(['GSM3673551', 'GSM3673552', 'GSM3673553', 'GSM3673554', 'GSM3673555'], dtype='object')


### Step 7: Data Normalization and Linking

In [8]:
# 1. We already normalized gene symbols in the gene expression data in step 6
# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First, let's extract the clinical features properly
# Based on the sample characteristics dictionary from step 1:
# {0: ['tissue: Blood'], 1: ['illness: Major Depressive Disorder'], 2: ['age: 16', 'age: 13', 'age: 12', 'age: 14', 'age: 17', 'age: 15'], 3: ['Sex: female', 'Sex: male']}

def convert_trait(value):
    """Convert depression status to binary format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "major depressive disorder" in value:
        return 1  # Has depression
    return 0  # Control/no depression

def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# Get clinical data using the correct row index identified in step 1
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Using row 1 for depression status (major depressive disorder)
    convert_trait=convert_trait,
    age_row=2,    # Age data is in row 2
    convert_age=convert_age,
    gender_row=3, # Gender data is in row 3
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples of children and adolescents with Major Depressive Disorder, before and after Fluoxetine treatment."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene expression data saved to ../../output/preprocess/Depression/gene_data/GSE128387.csv
Selected clinical data shape: (3, 32)
Clinical data preview:
{'GSM3673551': [1.0, 16.0, 0.0], 'GSM3673552': [1.0, 13.0, 0.0], 'GSM3673553': [1.0, 16.0, 0.0], 'GSM3673554': [1.0, 12.0, 0.0], 'GSM3673555': [1.0, 16.0, 0.0], 'GSM3673556': [1.0, 16.0, 0.0], 'GSM3673557': [1.0, 14.0, 0.0], 'GSM3673558': [1.0, 13.0, 0.0], 'GSM3673559': [1.0, 16.0, 0.0], 'GSM3673560': [1.0, 17.0, 0.0], 'GSM3673561': [1.0, 13.0, 0.0], 'GSM3673562': [1.0, 16.0, 0.0], 'GSM3673563': [1.0, 15.0, 1.0], 'GSM3673564': [1.0, 15.0, 0.0], 'GSM3673565': [1.0, 15.0, 0.0], 'GSM3673566': [1.0, 14.0, 0.0], 'GSM3673567': [1.0, 16.0, 0.0], 'GSM3673568': [1.0, 13.0, 0.0], 'GSM3673569': [1.0, 16.0, 0.0], 'GSM3673570': [1.0, 12.0, 0.0], 'GSM3673571': [1.0, 16.0, 0.0], 'GSM3673572': [1.0, 16.0, 0.0], 'GSM3673573': [1.0, 14.0, 0.0], 'GSM3673574': [1.0, 13.0, 0.0], 'GSM3673575': [1.0, 16.0, 0.0], 'GSM3673576': [1.0, 17.0, 0.0], 'GSM3673577': [1.

Data shape after handling missing values: (32, 23277)
Quartiles for 'Depression':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Depression' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 13.75
  50% (Median): 15.0
  75%: 16.0
Min: 12.0
Max: 17.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 3 occurrences. This represents 9.38% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



A new JSON file was created at: ../../output/preprocess/Depression/cohort_info.json
Dataset is not usable for analysis. No linked data file saved.
